In [4]:
import pandas as pd
import glob
import os
import math
from scipy.stats import norm

files = sorted(glob.glob("1*.csv"))
files = [f for f in files if "_climate_sigma" not in f]

def black_scholes_call(S, K, r, sigma, T):
    d1 = (math.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*math.sqrt(T))
    d2 = d1 - sigma*math.sqrt(T)
    return S*norm.cdf(d1) - K*math.exp(-r*T)*norm.cdf(d2)

for file in files:

    company_name = os.path.basename(file).replace(".csv", "")
    display_name = company_name[1:]   # remove leading "1"

    current_data = pd.read_csv(file)
    current_data['Return'] = current_data['Close'].pct_change()
    current_data['Return'] = current_data['Return'].apply(
        lambda x: math.log(1+x) if pd.notnull(x) else x
    )

    sigma_standard = current_data['Return'].std() * math.sqrt(252)

    climate_file = f"{company_name}_climate_sigma.csv"
    sigma_climate = pd.read_csv(climate_file)['ClimateSigma'].iloc[0]

    S0 = current_data['Close'].iloc[-1]
    K = S0
    r = 0.035
    T = 1

    price_standard = black_scholes_call(S0, K, r, sigma_standard, T)
    price_climate  = black_scholes_call(S0, K, r, sigma_climate, T)

    print(f"\n====== {display_name} ======")
    print("Standard Sigma:", sigma_standard)
    print("Climate Sigma:", sigma_climate)
    print("Vanilla Option Price:", price_standard)
    print("Climate-Based Option Price:", price_climate)
    print("Difference:", price_climate - price_standard)


====== amazon_us_d ======
Standard Sigma: 0.5450241014123978
Climate Sigma: 0.2144819849428176
Vanilla Option Price: 46.860596352403334
Climate-Based Option Price: 20.94927480615432
Difference: -25.911321546249013

====== apple_us_d ======
Standard Sigma: 0.43222692340396085
Climate Sigma: 0.2605424193429639
Vanilla Option Price: 49.05660664616455
Climate-Based Option Price: 31.660313985155085
Difference: -17.39629266100947

====== bp_us_d ======
Standard Sigma: 0.3084449229231894
Climate Sigma: 0.2881419869360738
Vanilla Option Price: 5.2704792572330135
Climate-Based Option Price: 4.972844648656835
Difference: -0.2976346085761783

====== chevron_us_d ======
Standard Sigma: 0.26645099511800036
Climate Sigma: 0.2140668837097365
Vanilla Option Price: 22.631201990460227
Climate-Based Option Price: 18.897383747830006
Difference: -3.733818242630221

====== coke_us_d ======
Standard Sigma: 0.23401376972806803
Climate Sigma: 0.1634295904015624
Vanilla Option Price: 8.763081973317874
Climate-